In [20]:
!wget https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/reviews.txt

--2022-07-25 10:31:59--  https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M  --.-KB/s    in 0.1s    

2022-07-25 10:32:01 (282 MB/s) - ‘reviews.txt’ saved [33678267/33678267]



In [21]:
!wget https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/labels.txt 

--2022-07-25 10:32:01--  https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.01s   

2022-07-25 10:32:01 (17.2 MB/s) - ‘labels.txt’ saved [225000/225000]



# Import Dependencies

In [22]:
from string import punctuation
from collections import Counter
import numpy as np
import pandas as pd
import random

# Load in the Data

In [23]:
# Read data from text files
with open("/content/reviews.txt") as f:
  reviews = f.read()

with open("/content/labels.txt") as f:
  labels = f.read()

# Data Exploration

In [24]:
reviews[: 1000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

In [25]:
# Characters present in reviews 
set(reviews)

{'\n',
 ' ',
 '.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [26]:
labels[: 1000]

'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\

# Data Preprocessing

In [27]:
# Convert alphabets to lowercase
text = reviews.lower()

# Remove the punctuations
text = ''.join([char  for char in text  if char not in punctuation])

# Remove '\n' character
reviews_split = text.split('\n') # List of reviews
text = ' '.join(reviews_split)

text[: 5000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [28]:
labels = labels.split()
labels[: 10]

['positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [29]:
# Check if zero-length reviews are present
review_lengths = Counter(len(review)  for review in reviews_split)
print(f"Zero-length reviews: {review_lengths[0]}")

Zero-length reviews: 1


In [30]:
# Get index of reviews having non-zero length
non_zero_idxs = [idx  for idx, review in enumerate(reviews_split)  if len(review)!=0]

# Remove zero-length reviews and their corresponding labels
reviews_split = [reviews_split[idx]  for idx in non_zero_idxs]
labels = [labels[idx]  for idx in non_zero_idxs]

In [31]:
print(f"No. of reviews: {len(reviews_split)}")
print(f"No. of labels : {len(labels)}")

No. of reviews: 25000
No. of labels : 25000


In [32]:
# Get a list of words used in reviews
words = text.split()
print(words[: 15])
print(f"\nThere are {len(words)} words in reviews.")

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other']

There are 6020196 words in reviews.


In [33]:
# Get vocabulary
vocabulary = set(words)
print(f"There are {len(vocabulary)} words in the vocabulary.")

There are 74072 words in the vocabulary.


### Create Dictionary


The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

    Exercise: Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers start at 1, not 0. Also, convert the reviews to integers and store the reviews in a new list called reviews_ints.


In [34]:
# Encode the words to integers
# The integers start from 1 (and not 0) as we'll pad our input vectors with zeros later
vocab_to_int = {word: idx  for idx, word in enumerate(vocabulary, 1)}


dict(list(vocab_to_int.items())[:10])

{'canoeing': 2,
 'diagram': 10,
 'franklyn': 9,
 'gratification': 5,
 'hypothetically': 6,
 'incredible': 8,
 'marti': 1,
 'pleasers': 4,
 'principal': 3,
 'windy': 7}

In [35]:
# int_vocab = dict(enumerate(unique_words))

# dict(list(int_vocab.items())[:10])

### Tokenize Reviews

In [36]:
reviews_int = [] # Will store tokenized reviews

# Tokenize each review
for review in reviews_split:
  reviews_int.append([vocab_to_int[word]  for word in review.split()])

In [37]:
rand_idx = random.randint(0, len(reviews_split))

print(f"REVIEW:\n   {reviews_split[rand_idx]}\n")
print(f"   No. of words in the review: {len(reviews_split[rand_idx].split())}\n")

print("-" * 20)

print(f"\nTOKENIZED VERSION:\n   {reviews_int[rand_idx]}\n")
print(f"   No. of tokens in the tokenized version: {len(reviews_int[rand_idx])}")


REVIEW:
   i was blown away by the re  imagined battlestar galactica  a show that always kept me guessing and brought me to tears on more than one occasion  a hardened sci  fi fan  i like to think i can pick out the good stuff from the bs  and this was good stuff   br    br   as such  when i first heard about the prospect of a prequel series some months ago i got a sick feeling in my gut  i was afraid that the formula that made battlestar so successful would be reused in caprica  which wouldn  t work at all  bsg  s story  of a mournful ragged band of survivors  trapped aboard decaying star ships and guided by prophetic vision and a sequence of pseudo  miracles  was perfectly complimented by extraordinary music and a better cast of actors   br    br   caprica feels different  where bsg takes place after the fall of a great civilization  caprica portrays that civilization in it  s cold and decadent heyday  the overall vibe i got from caprica was similar to that of minority report  minus 

### Encode Labels

In [50]:
encoded_labels = np.array([1 if label=="positive" else 0  for label in labels])
print(f"No. of labels: {len(encoded_labels)}")
print(encoded_labels[:20])

No. of labels: 25000
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


In [39]:
# Get length of the longest review
max_len = max(len(review)  for review in reviews_int)

# Get the average length of reviews
avg_len = sum([len(review)  for review in reviews_int]) / len(reviews_int)

print(f"The length of the longest review is {max_len}.")
print(f"The average length of reviews is {avg_len:.2f}")

The length of the longest review is 2514.
The average length of reviews is 240.81


In [40]:
def pad_reviews_with_zeros(reviews_int, seq_len):
  features = np.zeros((len(reviews_int), seq_len), dtype=np.int32)

  for idx, review in enumerate(reviews_int):
    features[idx][-len(review):] = np.array(review)[:seq_len] 

  return features

In [41]:
features = pad_reviews_with_zeros(reviews_int, seq_len=512)
features

array([[    0,     0,     0, ..., 37110,  7673,  6933],
       [    0,     0,     0, ..., 71387, 67335, 53822],
       [    0,     0,     0, ...,  1762, 58167, 32187],
       ...,
       [    0,     0,     0, ...,  8032, 54418, 21866],
       [    0,     0,     0, ..., 49017, 56340, 44760],
       [    0,     0,     0, ..., 10480, 53783, 20654]], dtype=int32)

In [42]:
type(encoded_labels)

numpy.ndarray

In [43]:
features.shape, encoded_labels.shape

((25000, 512), (25000,))

In [61]:
import tensorflow as tf

In [45]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels,
                                                    test_size=0.2,
                                                    shuffle=True)


print(f"""
X_train shape: {X_train.shape}
y_train shape: {y_train.shape}
X_test shape : {X_test.shape}
y_test shape : {y_test.shape}""")


X_train shape: (20000, 512)
y_train shape: (20000,)
X_test shape : (5000, 512)
y_test shape : (5000,)


In [59]:
one, zero = 1, 0

for  i in y_train[:100]:
  if i == 1:
    one = one+1
  else:
    zero = zero+1

print(one,zero)

49 52


In [60]:
y_test[:20]

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1])

### Generate Datasets

In [64]:
def get_training_dataset(X_train, y_train, batch_size=32):
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  train_dataset = train_dataset.shuffle(buffer_size=len(X_train), reshuffle_each_iteration=True)
  train_dataset = train_dataset.batch(batch_size)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

  return train_dataset

In [65]:
def get_validation_dataset(X_test, y_test, batch_size=32):
  validation_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  validation_dataset = validation_dataset.batch(batch_size)

  return validation_dataset

In [66]:
# Get training dataset
train_dataset = get_training_dataset(X_train, y_train, batch_size=32)

# Get validaton dataset
validation_dataset = get_validation_dataset(X_test, y_test, batch_size=32)

In [79]:
# Get sample training dataset
sample_x, sample_y = train_dataset.as_numpy_iterator().next()

print(f'Input shape: {sample_x.shape}') 
print(f'Input: \n{sample_x}')
print("\n")
print(f'Label shape: {sample_y.shape}') 
print(f'Label: \n{sample_y}')

Input shape: (32, 512)
Input: 
[[    0     0     0 ... 61453 44267 26205]
 [10480  6753 71387 ... 53783 63525 44267]
 [    0     0     0 ... 20034 25915 25915]
 ...
 [    0     0     0 ... 25915 25915 52195]
 [    0     0     0 ... 52195 58715 69505]
 [    0     0     0 ... 61320  1762 31635]]


Label shape: (32,)
Label: 
[1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 1]


In [ ]:
# rev = [[12, 13, 15],
#        [45, 85, 64, 45, 63, 78, 45],
#        [16, 75, 96, 42]]